In [1]:
f = open("subject_01.mat")

In [2]:
import mne
from scipy.io import loadmat
import numpy as np

In [3]:
chnames = ['Fp1','Fp2','Fc5','Fz','Fc6','T7','Cz','T8','P7','P3','Pz','P4','P8','O1','Oz','O2','stim']
chtypes = ['eeg'] * 16 + ['stim']
info = mne.create_info(ch_names=chnames, sfreq=512,ch_types=chtypes, montage='standard_1020',verbose=False)
data = loadmat("subject_01.mat")["SIGNAL"]

print(data[0])
print(len(data[0]))

[     0.         -16639.796875     1075.2641602    7197.5185547
   1501.6668701   -4080.4909668     872.66534424  -7446.5708008
  -3852.1730957    1030.7530518   -8910.7548828     959.37817383
  -2181.5031738    8054.1430664  -10051.151367    13269.295898
 -19776.359375        0.              0.        ]
19


In [4]:
import matplotlib.pyplot as plt
for k in range(16):
    plt.plot([d[0] for d in data],[d[k] for d in data])
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
"""
Colonne 1 : timestamp
Colonne 2 à 17 : les données des 16 éléctrodes
Colonne 18 : quand on passe aux yeux sont fermé
Colone 19 : quand on passe aux yeux sont ouvert
filled with zeros, except at the timestamp corresponding to the beginning of
the block for condition 1 (resp. 2), when the row gets a value of one.
"""

'\nColonne 1 : timestamp\nColonne 2 à 17 : les données des 16 éléctrodes\nColonne 18 : quand on passe aux yeux sont fermé\nColone 19 : quand on passe aux yeux sont ouvert\nfilled with zeros, except at the timestamp corresponding to the beginning of\nthe block for condition 1 (resp. 2), when the row gets a value of one.\n'

In [6]:
#On localise les indices des 1
print("0 : Je ferme les yeux")
for k in range(len(data)):
    if(data[k][17]==1):
        print(str(k)+" : J'ouvre les yeux")
    if(data[k][18]==1):
        print(str(k)+" : Je ferme les yeux")

0 : Je ferme les yeux
35511 : J'ouvre les yeux
45179 : Je ferme les yeux
53413 : J'ouvre les yeux
61886 : Je ferme les yeux
69812 : J'ouvre les yeux
78140 : Je ferme les yeux
85802 : J'ouvre les yeux
93556 : Je ferme les yeux
101416 : J'ouvre les yeux
109193 : Je ferme les yeux


In [7]:
#On modifie pour avoir un dataset avec 1 : yeux ouvert et -1 yeux fermée
new_data=[]
yeux_ouvert=False
for k in range(len(data)):
    if(yeux_ouvert==False):
        tmp=np.append(data[k][1:17],-1)
    else:
        tmp=np.append(data[k][1:17],1)
    new_data.append(tmp)
    if(data[k][17]==1):
        yeux_ouvert=True
    if(data[k][18]==1):
        yeux_ouvert=False

In [8]:
print(len(new_data))
print(len(new_data[0]))

119808
17


In [9]:
nbOuvert=0
nbFermee=0
ens_yeux_fermee=[]
ens_yeux_ouvert=[]
for d in new_data:
    if(d[16]==1):
        nbOuvert+=1
        ens_yeux_ouvert.append(d)
    elif(d[16]==-1):
        nbFermee+=1
        ens_yeux_fermee.append(d)
print(nbOuvert)
print(nbFermee)

42000
77808


In [10]:
min_sample=min(nbOuvert,nbFermee)
print(min_sample)

42000


In [11]:
np.random.shuffle(ens_yeux_fermee)
np.random.shuffle(ens_yeux_ouvert)
ens_yeux_fermee = ens_yeux_fermee[:min_sample]
ens_yeux_ouvert = ens_yeux_ouvert[:min_sample]


In [12]:
print(len(ens_yeux_fermee))
print(len(ens_yeux_ouvert))

42000
42000


In [13]:
data = ens_yeux_fermee+ens_yeux_ouvert
np.random.shuffle(data)
prop=0.8
ens_train=data[:int(len(data)*prop)]
ens_test=data[int(len(data)*prop):]

In [14]:
def pred(w,data):
    somme=0
    for k in range(len(w)):
        somme+=w[k]*data[k]
    return somme

In [30]:
def miseAjoutPoid(w,d,l):
    pas=0.0001
    for k in range(len(w)):
        w[k] = w[k] - pas*-l*d[k]
    return w

In [39]:
def apprentissage(batch_data,batch_label,w):
    for k in range(len(batch_data)):
        prediction = pred(w,batch_data[k])
        if(prediction*batch_label[k]<0):
            w=miseAjoutPoid(w,batch_data[k],batch_label[k])            

In [40]:
labels_train = [d[16] for d in ens_train]
donnees_train = [d[:16] for d in ens_train]

In [41]:
W = 2*np.random.random_sample((16))-1
print(W)
for _ in range(10):
    apprentissage(donnees_train,labels_train,W)
print(W)

[-0.15917372  0.12842456 -0.8863346  -0.00148862  0.9829083   0.76941328
 -0.71241396 -0.01853682 -0.69131347 -0.94343907 -0.06491946  0.37231047
  0.49430515  0.92979187  0.42557949  0.5982523 ]
[  28.01238387  305.32902256  -30.22724104  229.25181335   59.46232674
 -137.57564363   15.73506787  178.22124904  -56.68979086 -263.83675577
   74.40879907   27.34537204   67.01957168  164.97131903 -108.69763211
  -65.02161466]


In [42]:
labels_test = [d[16] for d in ens_test]
donnees_test = [d[:16] for d in ens_test]
prediction_test = pred(W,donnees_test)
bonnePred=0
for k in range(len(ens_test)):
    y_pred = pred(W,donnees_test[k])
    y_reel = labels_test[k]
    if((y_pred >0 and y_reel==1) or (y_pred<0 and y_reel==-1) ):
        bonnePred+=1
print(bonnePred/len(ens_test))

0.8432738095238095
